# Исследование клиентов для телекоммуникационной компании.

Заказчик — большая телекоммуникационная компания.
Цель: Перед компанией стоит задача определить текущий уровень потребительской лояльности (NPS) среди клиентов из России.
Задачи: 
1. Обработать материалы опроса;
2. Подготовка дашборда с его итогами. 
Работа выполняется в SQLite.
В рамках проекта необходимо выяснить:
Как распределены участники опроса по возрасту, полу и возрасту;
Соотношение количества старых и новых пользователей;
Активность населения городов в рамках опроса;
Степень лояльности разных групп пользователей сервису;
Общий NPS среди всех опрошенных;
Черты и особенности, характеризующие группу cторонников (promoters).

Структура проекта:

1. [Изучение данных.](#start)
2. [Предобработка данных.](#preprocessing)
3. [Создание и выгрузка таблицы.](#table)
4. [Общий вывод.](#outcome)

In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine


In [2]:
path_to_db_server = '/datasets/telecomm_csi.db'  

    
engine = create_engine(f'sqlite:///{path_to_db_server}', echo = False)

## Формирование запроса и изучение данных.<a id="start"></a>

In [3]:
query = """
SELECT u.user_id, u.lt_day,
       CASE
       WHEN u.lt_day <= 365 THEN 'TRUE'
       ELSE 'FALSE'
       END AS is_new,
u.age,
       CASE
       WHEN gender_segment = 0 THEN 'мужчина'
       ELSE 'женщина'
       END AS gender_segment,
u.os_name, u.cpe_type_name, l.country, l.city, SUBSTR(a_s.title, 4) AS age_segment,
SUBSTR(ts.title, 4) AS traffic_segment, SUBSTR(lts.title, 4) AS lifetime_segment,
u.nps_score,
        CASE
        WHEN u.nps_score <= 6 THEN 'detractors'
        WHEN u.nps_score >= 9 THEN 'promoters'
        ELSE 'passives'
        END AS nps_group
FROM location AS l
LEFT OUTER JOIN user AS u ON l.location_id = u.location_id
LEFT OUTER JOIN age_segment AS a_s ON a_s.age_gr_id = u.age_gr_id
LEFT OUTER JOIN traffic_segment AS ts ON ts.tr_gr_id = u.tr_gr_id
LEFT OUTER JOIN lifetime_segment AS lts ON lts.lt_gr_id = u.lt_gr_id

;
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A0CMN5,2729,FALSE,27.0,мужчина,ANDROID,SMARTPHONE,Россия,Архангельск,25-34,60-65,36+,6,detractors
1,A0CYVK,478,FALSE,48.0,женщина,ANDROID,SMARTPHONE,Россия,Архангельск,45-54,10-15,13-24,7,passives
2,A0HNQU,413,FALSE,42.0,женщина,ANDROID,SMARTPHONE,Россия,Архангельск,35-44,20-25,13-24,9,promoters


## Предобработка данных.<a id="preprocessing"></a>

In [5]:
display(df.isna().sum())
display(df.duplicated().sum())
display(df['age'].count())
display(df['gender_segment'].unique())
display(df['gender_segment'].value_counts())

user_id               0
lt_day                0
is_new                0
age                 554
gender_segment        0
os_name               0
cpe_type_name         0
country               0
city                  0
age_segment           0
traffic_segment       0
lifetime_segment      0
nps_score             0
nps_group             0
dtype: int64

0

501939

array(['мужчина', 'женщина'], dtype=object)

женщина    273743
мужчина    228750
Name: gender_segment, dtype: int64

Дубликатов строк в таблице нет. В колонке age присутствует 554 значения NULL из 502493 значений, что состаляет 0.11% от общего количества. Следовательно, мы можем пренебречь количеством значений NULL, как незначительным. В столбце gender_segment до применения CASE присутствуют значения 1, 0 и nan. Здесь присутствует 1301 значение nan, что составляет 0.26% от общего количества. Следовательно, мы можем пренебречь количеством значений nan, как незначительным. Выделять их в отдельную группунет необходимости.

## Создание и выгрузка таблицы.<a id="table"></a>

In [6]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


## Вывод.<a id="outcome"></a>

В результате исследования можно сделать следующие выводы:

Общий NPS всех клиентов компании составил 21.90%.

Медианный возраст клиентов компании - 39 лет. Медианный возраст для возрастных сегментов: до 16 лет - 14; 16-24 года - 22; 25-34 года - 31; 35-44 года - 39; 45-54 года - 49; 55-64 года - 58; 66 + лет - 68.

Медианный возраст с учётом гендерной принадлежности не изменяется - 39 лет, как у мужчин, так и у женщин. В межквартильном интервале женщины на полгода старше мужчин.

Новые клиенты составляют 17.13% от общего количества, старые - 82.87. NPS у "новичков" значительно выше - 35.62 против 19.06 у "старых" клиентов.

Больше всего участников опроса проживает в Москве, Санкт-Петербурге и в других городах-миллионниках. При этом количество участников в миллионниках уменьшается непропорционально уменьшению населения. Определить именно степень активности в каждом городе невозможно, поскольку нет данных о населении каждого города. Случайные внешние данные позволяют говорить о том, что активность томичей в 2, а новосибирцев - приблизительно в 3 раза выше, чем москвичей.

Есть определённая зависимость степени лояльности клиентов от возраста. С 16 до 66 лет наблюдается рост лояльности. В каждой возрастной группе женщины значительно более лояльны компании, чем мужчины. В группах 16-24 и 25-34 года уровень лояльности среди женщин почти в 2 раза выше, чем среди мужчин. Не лояльны только возрастная группа до 16 лет и людя, не указавшие возраст.

В группах по типу устройства наиболее лояльны (100%) владельцы MIFI-роутеров. Владельцы WLAN-роутеров, портативных устройств и карманных компьютеров демонстрируют нулевое значение NPS, владельцы модемов и mobile test platform - отрицательное значение NPS. Крайними значениями, как и нулевыми, можно пренебречь, поскольку их владельцы представленны числом от 1 до 7. Самая многочисленная группа (владельцы смртфонов) демонстрируют NPS равный 21.5.

Наибольшую лояльность в группах по объёму трафика показывают наименее активные пользователи с трафиком до 1 мегабайта (34.62%, 28.21%, 26.94%). Наименьший уровень лояльности свойствен самой активной группе с трафиком больше 24100 мегабайт (2.48%). При этом, самая многочисленная группа (112649 человек), объём трафика которой равен от 1 до 5 мегабайт, показывает уровень лояльности 22.88%.

Уровень лояльности опрошенных не зависит от места проживания. Нет зависимости от проживания в столице, городах миллионниках или в городах среднего размера.

В зависимости от используемой операционной системы, наиболее лояльными оказались клиенты, использующие SYMBIAN OS. Далее с небольшими интервалами идут BADA OS, варианты WINDOWS, ANDROID и, с большим отрывом - IOS, пользователи которой наименее лояльны.

Группа promoters составляет большинство среди клиентов компании - 52.55%
В группе promoters пользователи предпочитают OS ANDROID (88.88%). Близкие по значению показатели свойственны и для двух других групп. IOS используют 9.66% promoters, что приблизительно в полтора раза меньше этого показателя для двух других групп.
96.71% promoters предпочитают использовать смартфоны. Близкие по значению показатели свойственны представителям двух других групп.
В большей части сегментов по потреблению трафика promoters составляют заметное большинство. Больше всего их количественно среди тех, кто использует до 10 мегабайт.
Основную часть группы promoters составляют мужчины и женщины в возрасте 25-44 года. Причём, женщины в обеих возрастных группах преобладают. В группе 35-44 года, такое преобладание очень заметно.
Таким образом, в группе promoters, составляющей 52.55% всех клиентов, представлены, как мужчины, так и женщины всех возрастных групп. Возрас основного количества клиентов - 25-44 года. Женщин реди них заметно больше. Представители этой группы используют трафик объёме до 10 мегабайт. Они предпочитают использовать смартфоны на OS ANDROID, не отличаясь в этом от представителей двух других групп клиентов компании.

### Ссылка на дашборд на сайте Tableau Public:

https://public.tableau.com/app/profile/sergei.kinev/viz/_16563355732310/promoters_5?publish=yes

### Ссылка на pdf-файл с презентацией:

https://drive.google.com/file/d/1QAZQUPV2uxqh7GXBh2RrrRXCj_EeL0zq/view?usp=sharing